In [1]:
import os
import numpy as np
import pandas as pd
import json
import pprint
import glob
import datetime
import random
import re
import warnings
from sklearn.impute import KNNImputer

warnings.filterwarnings('ignore')


# Loading Files:

#### Loading Accident Data:

In [ ]:
# Base directory where the files are 
base_direct = "~/data/FARS_Data/"

# Empty dictionary which stores every year's df
acc_data = {}

# Add data from CSV files from 1975 to 2021 
for year in range(1975, 2022):
    
    # Create the year's file path
    file_path = base_direct + f"ACCIDENT_{year}.csv"
    
    # ISO-8859-1 encoding is the type for these CSV files:
    acc_data[f'{year}'] = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)



In [3]:
len(acc_data)

47

In [4]:
acc_data['2021'].shape

(39508, 80)

In [5]:
acc_data['1975'].head()

,STATE,COUNTY,MONTH,DAY,YEAR,HOUR,MINUTE,VE_FORMS,PERSONS,VEHICLES,...,SCH_BUS,CF1,CF2,CF3,FATALS,DAY_WEEK,DRUNK_DR,ST_CASE,CITY,RAIL
0,1,95,1,4,75,22,24,3,3,NaN,...,NaN,0,0,0,1,7,0,10001,400,*******
1,1,95,1,2,75,4,30,1,2,NaN,...,NaN,0,0,0,1,5,0,10002,110,*******
2,1,35,1,1,75,4,0,1,1,NaN,...,NaN,0,0,0,1,4,0,10003,0,*******
3,1,77,1,2,75,14,50,1,3,NaN,...,NaN,0,0,0,1,5,0,10004,0,*******
4,1,97,1,5,75,20,0,1,1,NaN,...,NaN,0,0,0,1,1,0,10005,2100,*******


In [6]:
type(acc_data['1975'].head())

pandas.core.frame.DataFrame

#### Joining the 3 accident, person, and vehicle dataframes for each year isn't working since it takes a very long time to join everything & remove duplicates. Will need to create models for each dataset separately

# The EDA Steps Are:

1. Filtering unnecessary columns
2. Counting NAs in each column & imputing them
3. Rewording and simplifying column values
4. Summary statistics of quantitative columns

#### Filtering Unnecessary Columns:

Need to exclude columns that assign numbers to diff categories. For instance, numbers are assigned to each state, but this numerical column should be excluded since the states have no hierarchy to them.

In [7]:
acc_data['2021'].columns

Index(['STATE', 'STATENAME', 'ST_CASE', 'PEDS', 'PERNOTMVIT', 'VE_TOTAL',
       'VE_FORMS', 'PVH_INVL', 'PERSONS', 'PERMVIT', 'COUNTY', 'COUNTYNAME',
       'CITY', 'CITYNAME', 'MONTH', 'MONTHNAME', 'DAY', 'DAYNAME', 'DAY_WEEK',
       'DAY_WEEKNAME', 'YEAR', 'HOUR', 'HOURNAME', 'MINUTE', 'MINUTENAME',
       'TWAY_ID', 'TWAY_ID2', 'ROUTE', 'ROUTENAME', 'RUR_URB', 'RUR_URBNAME',
       'FUNC_SYS', 'FUNC_SYSNAME', 'RD_OWNER', 'RD_OWNERNAME', 'NHS',
       'NHSNAME', 'SP_JUR', 'SP_JURNAME', 'MILEPT', 'MILEPTNAME', 'LATITUDE',
       'LATITUDENAME', 'LONGITUD', 'LONGITUDNAME', 'HARM_EV', 'HARM_EVNAME',
       'MAN_COLL', 'MAN_COLLNAME', 'RELJCT1', 'RELJCT1NAME', 'RELJCT2',
       'RELJCT2NAME', 'TYP_INT', 'TYP_INTNAME', 'REL_ROAD', 'REL_ROADNAME',
       'WRK_ZONE', 'WRK_ZONENAME', 'LGT_COND', 'LGT_CONDNAME', 'WEATHER',
       'WEATHERNAME', 'SCH_BUS', 'SCH_BUSNAME', 'RAIL', 'RAILNAME', 'NOT_HOUR',
       'NOT_HOURNAME', 'NOT_MIN', 'NOT_MINNAME', 'ARR_HOUR', 'ARR_HOURNAME',
       'ARR_M

In [8]:
acc_columns = set(acc_data['1975'].columns)


In [9]:
# Subsetting relevant 29 columns from acc_data:

acc_relv_cols = ['STATENAME', 'ST_CASE', 'PEDS', 'VE_TOTAL', 'PERSONS', 'COUNTYNAME', 'CITYNAME', 
                 'MONTHNAME', 'DAYNAME', 'DAY_WEEKNAME', 'HOUR', 'TWAY_ID', 'ROUTENAME', 'RUR_URBNAME', 
                 'FUNC_SYSNAME', 'RD_OWNERNAME', 'MILEPT', 'LATITUDE', 'LONGITUD', 'HARM_EVNAME', 
                 'MAN_COLLNAME', 'TYP_INTNAME', 'REL_ROADNAME', 'WRK_ZONENAME', 'LGT_CONDNAME', 
                 'WEATHERNAME', 'SCH_BUS', 'FATALS']

# Checking if each relevant column is present within each dataframe since older year
# files don't have many columns present in newer year files

# The variable "cmn_cols" only has relevant columns that are part of the dataframe 
# and stores them in a dictionary called: subset_acc_data

subset_acc_data = {}

for key, df in acc_data.items():
    cmn_cols = [col for col in acc_relv_cols if col in df.columns]
    subset_acc_data[key] = df[cmn_cols]



In [10]:
len(subset_acc_data)

47

In [11]:
type(subset_acc_data)

dict

It seems that not all of the 29 relevant columns are in the earlier years of accident data. To address this, different models will be trained for different time periods & compared accordingly to assess changes in the dependent variable based on different "time periods" such as:

Time Period 1: 1975 - 1981 all have 5 columns

Time Period 2: 1982 - 1990 all have 7 columns

Time Period 3: 1991 - 2000 all have 8 columns (1999 & 2000 have 10 columns but will be reduced to 8)

Time Period 4: 2001 - 2007 (2001 to 2004 has 8 columns & 2005 to 2007 has 9, but these will be reduced to 8 colums for consistency in the time period)

Time Period 5: 2008 - 2014 all have 11 columns

Time Period 6: 2015 - 2021 all have 28 columns except 2015 that has 26, but all of its values will be marked as missing or imputed so that the other columns' values are preserved

#### Time Period 1 - EDA & Data Cleaning to Prep for Modeling:

In [13]:
subset_acc_data['1975'].head()

,ST_CASE,PERSONS,HOUR,SCH_BUS,FATALS
0,10001,3,22,NaN,1
1,10002,2,4,NaN,1
2,10003,1,4,NaN,1
3,10004,3,14,NaN,1
4,10005,1,20,NaN,1


In [14]:
acc_dfs_section_1 = {
    '1975': subset_acc_data['1975'],
    '1976': subset_acc_data['1976'],
    '1977': subset_acc_data['1977'],
    '1978': subset_acc_data['1978'],
    '1979': subset_acc_data['1979'],
    '1980': subset_acc_data['1980'],
    '1981': subset_acc_data['1981']   
    
}


In [15]:
# Each year's df's number of NAs
na_counts_dict_sect_1 = {}

# Storing every column's NA counts in a dictionary
for yr, df in acc_dfs_section_1.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_1[yr] = na_counts

# Print year's columns' NA counts
for yr, na_counts in na_counts_dict_sect_1.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 1975:
ST_CASE        0
PERSONS        0
HOUR           0
SCH_BUS    39161
FATALS         0
dtype: int64
Year 1976:
ST_CASE        0
PERSONS        0
HOUR           0
SCH_BUS    39747
FATALS         0
dtype: int64
Year 1977:
ST_CASE    0
PERSONS    0
HOUR       0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1978:
ST_CASE    0
PERSONS    0
HOUR       0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1979:
ST_CASE    0
PERSONS    0
HOUR       0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1980:
ST_CASE    0
PERSONS    0
HOUR       0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1981:
ST_CASE    0
PERSONS    0
HOUR       0
SCH_BUS    0
FATALS     0
dtype: int64


In [16]:
acc_dfs_section_1['1975']['SCH_BUS']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
39156   NaN
39157   NaN
39158   NaN
39159   NaN
39160   NaN
Name: SCH_BUS, Length: 39161, dtype: float64

SCH_BUS values are only present for 1977 data onwards but is completely missing for 1975 and 1976. The NA values in 1975 and 1976 will be imputed to be 0, meaning a school bus was not involved in the fatalities.

In [17]:
acc_dfs_section_1['1975'].loc[:, 'SCH_BUS'] = acc_dfs_section_1['1975']['SCH_BUS'].fillna(0)
acc_dfs_section_1['1975'].head()

,ST_CASE,PERSONS,HOUR,SCH_BUS,FATALS
0,10001,3,22,0.0,1
1,10002,2,4,0.0,1
2,10003,1,4,0.0,1
3,10004,3,14,0.0,1
4,10005,1,20,0.0,1


In [18]:
acc_dfs_section_1['1976'].loc[:, 'SCH_BUS'] = acc_dfs_section_1['1976']['SCH_BUS'].fillna(0)
acc_dfs_section_1['1976'].head()

,ST_CASE,PERSONS,HOUR,SCH_BUS,FATALS
0,10001,2,17,0.0,1
1,10002,2,13,0.0,1
2,10003,2,20,0.0,1
3,10004,5,9,0.0,3
4,10005,2,8,0.0,1


#### Time Period 2 - EDA & Data Cleaning to Prep for Modeling:
Time Period 2: 1982 - 1990 all have 7 columns

In [19]:
subset_acc_data['1982'].head()

,ST_CASE,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,2,8,65,2131.0,0,2
1,10002,1,23,0409,0.0,0,1
2,10003,1,12,0234,0.0,0,1
3,10004,4,23,59,99999.0,0,2
4,10005,2,10,0310,0.0,0,1


In [20]:
acc_dfs_section_2 = {
    '1982': subset_acc_data['1982'],
    '1983': subset_acc_data['1983'],
    '1984': subset_acc_data['1984'],
    '1985': subset_acc_data['1985'],
    '1986': subset_acc_data['1986'],
    '1987': subset_acc_data['1987'],
    '1988': subset_acc_data['1988'],
    '1989': subset_acc_data['1989'],
    '1990': subset_acc_data['1990']
    
}


In [21]:
na_counts_dict_sect_2 = {}

for yr, df in acc_dfs_section_2.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_2[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_2.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 1982:
ST_CASE      0
PERSONS      0
HOUR         0
TWAY_ID     22
MILEPT     140
SCH_BUS      0
FATALS       0
dtype: int64
Year 1983:
ST_CASE      0
PERSONS      0
HOUR         0
TWAY_ID      5
MILEPT     121
SCH_BUS      0
FATALS       0
dtype: int64
Year 1984:
ST_CASE      0
PERSONS      0
HOUR         0
TWAY_ID     11
MILEPT     150
SCH_BUS      0
FATALS       0
dtype: int64
Year 1985:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    4
MILEPT     5
SCH_BUS    0
FATALS     0
dtype: int64
Year 1986:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1987:
ST_CASE     0
PERSONS     0
HOUR        0
TWAY_ID    12
MILEPT     12
SCH_BUS     0
FATALS      0
dtype: int64
Year 1988:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1989:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1990:
ST_CASE    0
PERSONS

The column TWAY_ID represents the road name where the accident occurred It may stand for through-way ID. Number values can be assumed to be the route number. For the relatively few missing values in some of the years' dataframes, the best is to replace NA values with "Missing" so that NA values don't interfere with model building.

Missing values of milepoint will be replaced with the mean & it will remain quantitative.

In [22]:
for year, df in acc_dfs_section_2.items():
    
    acc_dfs_section_2[year]['TWAY_ID'].fillna('Missing', inplace=True)


In [23]:
# Recheck the number of NA values:

for yr, df in acc_dfs_section_2.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_2[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_2.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 1982:
ST_CASE      0
PERSONS      0
HOUR         0
TWAY_ID      0
MILEPT     140
SCH_BUS      0
FATALS       0
dtype: int64
Year 1983:
ST_CASE      0
PERSONS      0
HOUR         0
TWAY_ID      0
MILEPT     121
SCH_BUS      0
FATALS       0
dtype: int64
Year 1984:
ST_CASE      0
PERSONS      0
HOUR         0
TWAY_ID      0
MILEPT     150
SCH_BUS      0
FATALS       0
dtype: int64
Year 1985:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     5
SCH_BUS    0
FATALS     0
dtype: int64
Year 1986:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1987:
ST_CASE     0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT     12
SCH_BUS     0
FATALS      0
dtype: int64
Year 1988:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1989:
ST_CASE    0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1990:
ST_CASE    0
PERSONS

In [24]:
subset_acc_data['1990'].tail(30)

,ST_CASE,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
39806,560077,2,14,258,151,0,1
39807,560078,3,21,487,305,0,1
39808,560079,2,0,120,579,0,1
39809,560080,3,1,16,550,0,2
39810,560081,4,2,I80,854,0,1
39811,560082,2,7,789,1043,0,1
39812,560083,3,9,14-87-I90B,252,0,1
39813,560084,1,16,191,5143,0,1
39814,560085,2,1,FAU4661,0,0,1
39815,560086,3,23,390,45,0,2


#### Time Period 3 - EDA & Data Cleaning to Prep for Modeling:

Time Period 3: 1991 - 2000 all have 8 columns (1999 & 2000 have 10 columns but will be reduced to 8)

In [25]:
subset_acc_data['1991'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,2,11,I20,1723,0,1
1,10002,0,6,7,SR13,25,0,2
2,10003,0,1,23,5849,0,0,1
3,10004,0,2,16,2208,0,0,1
4,10005,0,2,15,69,1882,0,1


In [26]:
# Years 1991 to 2000
acc_dfs_section_3 = {
    '1991': subset_acc_data['1991'],
    '1992': subset_acc_data['1992'],
    '1993': subset_acc_data['1993'],
    '1994': subset_acc_data['1994'],
    '1995': subset_acc_data['1995'],
    '1996': subset_acc_data['1996'],
    '1997': subset_acc_data['1997'],
    '1998': subset_acc_data['1998'],
    '1999': subset_acc_data['1999'],
    '2000': subset_acc_data['2000'],
}

In [27]:
na_counts_dict_sect_3 = {}

for yr, df in acc_dfs_section_3.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_3[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_3.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 1991:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1992:
ST_CASE     0
PEDS        0
PERSONS     0
HOUR        0
TWAY_ID    36
MILEPT     36
SCH_BUS     0
FATALS      0
dtype: int64
Year 1993:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    1
MILEPT     1
SCH_BUS    0
FATALS     0
dtype: int64
Year 1994:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1995:
ST_CASE     0
PEDS        0
PERSONS     0
HOUR        0
TWAY_ID    10
MILEPT      0
SCH_BUS     0
FATALS      0
dtype: int64
Year 1996:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1997:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1998:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0

In [28]:
# Remove 2 columns in 1999 & 2000 that are not present in the rest of this 
# time period's dataframes:

acc_dfs_section_3['1999'] = acc_dfs_section_3['1999'].drop(columns=['LATITUDE', 
                                                                    'LONGITUD'])

acc_dfs_section_3['2000'] = acc_dfs_section_3['2000'].drop(columns=['LATITUDE', 
                                                                    'LONGITUD'])

In [29]:
acc_dfs_section_3['1999'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,3,2,1028,0,0,1
1,10002,0,2,5,1040,0,0,1
2,10003,0,1,17,SR-265,12,0,1
3,10004,0,3,18,I-59,853,0,2
4,10005,0,1,20,1217,0,0,1


In [30]:
acc_dfs_section_3['2000'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,1,5,5,US-SR3,99999,0,1
1,10002,0,1,18,1112,0,0,1
2,10003,0,1,8,5115,0,0,1
3,10004,0,1,3,SR-22,625,0,1
4,10005,0,3,9,SR-113,89,0,2


The only values that are missing are for TWAY_ID & MILEPT.

In [31]:
# Imputing values for TWAY_ID & MILEPT:
for year, df in acc_dfs_section_3.items():
    
    acc_dfs_section_3[year]['TWAY_ID'].fillna('Missing', inplace=True)


In [32]:
# Recheck NA Values:

for yr, df in acc_dfs_section_3.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_3[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_3.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 1991:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1992:
ST_CASE     0
PEDS        0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT     36
SCH_BUS     0
FATALS      0
dtype: int64
Year 1993:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     1
SCH_BUS    0
FATALS     0
dtype: int64
Year 1994:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1995:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1996:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1997:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 1998:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS

In [33]:
subset_acc_data['2000'].tail(30)

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,LATITUDE,LONGITUD,SCH_BUS,FATALS
37496,560103,0,2,9,I-80,124,88888888,888888888,0,1
37497,560104,0,3,20,I-80,3172,88888888,888888888,0,1
37498,560105,0,3,20,US-26,1197,88888888,888888888,0,2
37499,560106,5,7,0,I-25,1859,88888888,888888888,0,2
37500,560107,1,7,13,SHIPTON LANE,0,88888888,888888888,0,1
37501,560108,1,2,0,UINTA DRIVE,0,88888888,888888888,0,1
37502,560109,1,6,19,SR-137,1001,88888888,888888888,0,1
37503,560110,0,1,20,SR-450,34,88888888,888888888,0,1
37504,560111,0,7,11,US-30,3152,88888888,888888888,0,1
37505,560112,0,4,17,US-16,2129,88888888,888888888,0,1


#### Time Period 4 - EDA & Data Cleaning to Prep for Modeling:

Time Period 4: 2001 - 2007 (2001 to 2004 has 8 columns & 2005 to 2007 has 9, but these will be reduced to 8 colums for consistency in the time period)

In [34]:
subset_acc_data['2001'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,1,20,1274,0.0,0,1
1,10002,0,1,5,SR-204,75.0,0,1
2,10003,0,2,22,SR-21,1230.0,0,1
3,10004,0,2,17,I-65,1642.0,0,1
4,10005,0,2,9,US-SR3,2993.0,0,1


In [35]:
subset_acc_data['2005'].head()

,ST_CASE,PEDS,VE_TOTAL,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,2,2,16,1172,0.0,0,1
1,10002,0,1,1,19,SR-69,2275.0,0,1
2,10003,0,1,1,11,I-20,1567.0,0,1
3,10004,0,1,1,13,1340,0.0,0,1
4,10005,0,1,1,5,1491,0.0,0,1


In [36]:
acc_dfs_section_4 = {
    '2001': subset_acc_data['2001'],
    '2002': subset_acc_data['2002'],
    '2003': subset_acc_data['2003'],
    '2004': subset_acc_data['2004'],
    '2005': subset_acc_data['2005'],
    '2006': subset_acc_data['2006'],
    '2007': subset_acc_data['2007']
}

In [37]:
# Remove the VE_TOTAL column in 2005, 2006 & 2007 that are not present in the rest 
# of this time period's dataframes:

for year in range(2005, 2008):

    acc_dfs_section_4[str(year)] = acc_dfs_section_4[str(year)].drop(columns=
                                                                     ['VE_TOTAL'])

In [38]:
acc_dfs_section_4['2007'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,1,23,1493,0.0,0,1
1,10002,0,4,13,SR-68,0.0,0,2
2,10003,0,5,12,US-SR53,0.0,0,1
3,10004,0,2,15,US-SR1,2468.0,0,1
4,10005,0,5,19,SR-14,0.0,0,1


In [39]:
na_counts_dict_sect_4 = {}

for yr, df in acc_dfs_section_4.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_4[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_4.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 2001:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 2002:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 2003:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 2004:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 2005:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 2006:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64
Year 2007:
ST_CASE    0
PEDS       0
PERSONS    0
HOUR       0
TWAY_ID    0
MILEPT     0
SCH_BUS    0
FATALS     0
dtype: int64


#### Time Period 5 - EDA & Data Cleaning to Prep for Modeling:
Time Period 5: 2008 - 2014 all have 11 columns

In [40]:
subset_acc_data['2008'].head()

,ST_CASE,PEDS,VE_TOTAL,PERSONS,HOUR,TWAY_ID,MILEPT,LATITUDE,LONGITUD,SCH_BUS,FATALS
0,10001,1,1,3,17,SR-69,0.0,34.326947,-86.489567,0,1
1,10002,0,2,2,12,I-20,1230.0,33.520883,-86.847739,0,1
2,10003,0,1,1,2,1027,0.0,32.477511,-86.346628,0,1
3,10004,0,1,2,10,8438,0.0,30.814786,-88.171917,0,1
4,10005,2,1,3,21,US-SR6,0.0,32.326347,-86.320975,0,2


In [41]:
acc_dfs_section_5 = {
    '2008': subset_acc_data['2008'],
    '2009': subset_acc_data['2009'],
    '2010': subset_acc_data['2010'],
    '2011': subset_acc_data['2011'],
    '2012': subset_acc_data['2012'],
    '2013': subset_acc_data['2013'],
    '2014': subset_acc_data['2014']
}

In [42]:
na_counts_dict_sect_5 = {}

for yr, df in acc_dfs_section_5.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_5[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_5.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 2008:
ST_CASE       0
PEDS          0
VE_TOTAL      0
PERSONS       0
HOUR          0
TWAY_ID       0
MILEPT        0
LATITUDE    483
LONGITUD    481
SCH_BUS       0
FATALS        0
dtype: int64
Year 2009:
ST_CASE       0
PEDS          0
VE_TOTAL      0
PERSONS       0
HOUR          0
TWAY_ID       0
MILEPT        0
LATITUDE    364
LONGITUD    363
SCH_BUS       0
FATALS        0
dtype: int64
Year 2010:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2011:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2012:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2013:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0

In [43]:
acc_dfs_section_5['2009']

,ST_CASE,PEDS,VE_TOTAL,PERSONS,HOUR,TWAY_ID,MILEPT,LATITUDE,LONGITUD,SCH_BUS,FATALS
0,10001,0,2,2,18,US-SR 5,1626.0,33.805147,-87.133511,0,1
1,10002,1,1,2,20,1480,0.0,30.603869,-87.864822,0,1
2,10003,0,2,3,15,SR-9,2228.0,33.693061,-85.676628,0,2
3,10004,0,2,3,8,SR-123,149.0,31.265603,-85.623536,0,1
4,10005,0,1,1,0,1611,0.0,34.989486,-87.225781,0,1
...,...,...,...,...,...,...,...,...,...,...,...
30857,560112,0,1,7,7,I-80,2898.0,41.441794,-105.950222,0,2
30858,560113,0,1,5,8,I-80,986.0,41.553353,-109.319386,0,1
30859,560114,0,2,5,21,SEVENTEEN-MILE RD,9.0,42.997214,-108.747819,0,2
30860,560115,0,1,1,5,TROUT CREEK RD,5.0,42.985956,-108.915344,0,1


#### KNN Imputation for Latitude & Longitude:
Because mean imputation would not make sense for geographical locations.

Chose number of neighbors to be 176 because it's the square root of the number of observations in the 2009 dataframe: sqrt(30,862) ~ 176. 

This is a logical value to start for the number of neighbors to impute just like the default value for mtry tends to be the square root of the number of observations.

In [44]:
# KNNImputer object
imputer = KNNImputer(n_neighbors=176) 

for year in range(2008, 2010):

    # Applying the KNNImputer object to missing latitdude & longitude values
    imputed_data = imputer.fit_transform(acc_dfs_section_5[str(year)][['LATITUDE', 
                                                                       'LONGITUD']])

    # Filling in NAs with imputed values
    acc_dfs_section_5[str(year)].loc[:, ['LATITUDE', 'LONGITUD']] = imputed_data


In [45]:
# Recheck NA values:

for yr, df in acc_dfs_section_5.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_5[yr] = na_counts


for yr, na_counts in na_counts_dict_sect_5.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 2008:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2009:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2010:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2011:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2012:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
LATITUDE    0
LONGITUD    0
SCH_BUS     0
FATALS      0
dtype: int64
Year 2013:
ST_CASE     0
PEDS        0
VE_TOTAL    0
PERSONS     0
HOUR        0
TWAY_ID     0
MILEPT      0
L

#### Time Period 6 - EDA & Data Cleaning to Prep for Modeling:
Time Period 6: 2015 - 2021 all have 29 columns except 2015 that has 27, but all of its values will be marked as missing or imputed so that the other columns' values are preserved

In [46]:
subset_acc_data['2015'].head()

,STATENAME,ST_CASE,PEDS,VE_TOTAL,PERSONS,MONTHNAME,DAYNAME,DAY_WEEKNAME,HOUR,TWAY_ID,...,LONGITUD,HARM_EVNAME,MAN_COLLNAME,TYP_INTNAME,REL_ROADNAME,WRK_ZONENAME,LGT_CONDNAME,WEATHERNAME,SCH_BUS,FATALS
0,Alabama,10001,0,1,1,January,1,Thursday,2,SR-5,...,-87.325328,Embankment,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear,0,1
1,Alabama,10002,0,1,1,January,1,Thursday,22,I-65,...,-86.908708,Ditch,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Median,None,Dark - Not Lighted,Cloudy,0,1
2,Alabama,10003,0,1,2,January,1,Thursday,1,US-SR 6,...,-85.758456,Tree (Standing Only),Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear,0,1
3,Alabama,10004,0,1,1,January,4,Sunday,0,SR-27,...,-85.510300,Mail Box,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Cloudy,0,1
4,Alabama,10005,0,2,2,January,7,Wednesday,7,US-SR 53,...,-85.515100,Motor Vehicle In-Transport,Angle,T-Intersection,On Roadway,None,Daylight,Clear,0,1


In [47]:
acc_dfs_section_6 = {
    '2015': subset_acc_data['2015'],
    '2016': subset_acc_data['2016'],
    '2017': subset_acc_data['2017'],
    '2018': subset_acc_data['2018'],
    '2019': subset_acc_data['2019'],
    '2020': subset_acc_data['2020'],
    '2021': subset_acc_data['2021']
}

No missing values found, except for 2 columns in the 2015 dataset, for which values will be filled in as missing:

In [48]:
columns_2015 = set(acc_dfs_section_6['2015'].columns)
columns_2016 = set(acc_dfs_section_6['2016'].columns)

columns_only_in_2016 = columns_2016 - columns_2015

print("Columns in 2016 but not in 2015:", columns_only_in_2016)


Columns in 2016 but not in 2015: {'CITYNAME', 'COUNTYNAME'}


Filling in missing columns in 2015 with 'Missing' so that these column values in years 2016 to 2021 are preserved for analyzing this time period:

In [49]:
acc_dfs_section_6['2015'].loc[:, 'COUNTYNAME'] = 'NOT AVAILABLE'
acc_dfs_section_6['2015'].loc[:, 'CITYNAME'] = 'NOT AVAILABLE'

In [50]:
acc_dfs_section_6['2015'].head()

,STATENAME,ST_CASE,PEDS,VE_TOTAL,PERSONS,MONTHNAME,DAYNAME,DAY_WEEKNAME,HOUR,TWAY_ID,...,MAN_COLLNAME,TYP_INTNAME,REL_ROADNAME,WRK_ZONENAME,LGT_CONDNAME,WEATHERNAME,SCH_BUS,FATALS,COUNTYNAME,CITYNAME
0,Alabama,10001,0,1,1,January,1,Thursday,2,SR-5,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear,0,1,NOT AVAILABLE,NOT AVAILABLE
1,Alabama,10002,0,1,1,January,1,Thursday,22,I-65,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Median,None,Dark - Not Lighted,Cloudy,0,1,NOT AVAILABLE,NOT AVAILABLE
2,Alabama,10003,0,1,2,January,1,Thursday,1,US-SR 6,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear,0,1,NOT AVAILABLE,NOT AVAILABLE
3,Alabama,10004,0,1,1,January,4,Sunday,0,SR-27,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Cloudy,0,1,NOT AVAILABLE,NOT AVAILABLE
4,Alabama,10005,0,2,2,January,7,Wednesday,7,US-SR 53,...,Angle,T-Intersection,On Roadway,None,Daylight,Clear,0,1,NOT AVAILABLE,NOT AVAILABLE


In [51]:
na_counts_dict_sect_6 = {}

for yr, df in acc_dfs_section_6.items():
    na_counts = df.isna().sum()
    na_counts_dict_sect_6[yr] = na_counts

for yr, na_counts in na_counts_dict_sect_6.items():
    print(f"Year {yr}:")
    print(na_counts)

Year 2015:
STATENAME       0
ST_CASE         0
PEDS            0
VE_TOTAL        0
PERSONS         0
MONTHNAME       0
DAYNAME         0
DAY_WEEKNAME    0
HOUR            0
TWAY_ID         0
ROUTENAME       0
RUR_URBNAME     0
FUNC_SYSNAME    0
RD_OWNERNAME    0
MILEPT          0
LATITUDE        0
LONGITUD        0
HARM_EVNAME     0
MAN_COLLNAME    0
TYP_INTNAME     0
REL_ROADNAME    0
WRK_ZONENAME    0
LGT_CONDNAME    0
WEATHERNAME     0
SCH_BUS         0
FATALS          0
COUNTYNAME      0
CITYNAME        0
dtype: int64
Year 2016:
STATENAME       0
ST_CASE         0
PEDS            0
VE_TOTAL        0
PERSONS         0
COUNTYNAME      0
CITYNAME        0
MONTHNAME       0
DAYNAME         0
DAY_WEEKNAME    0
HOUR            0
TWAY_ID         0
ROUTENAME       0
RUR_URBNAME     0
FUNC_SYSNAME    0
RD_OWNERNAME    0
MILEPT          0
LATITUDE        0
LONGITUD        0
HARM_EVNAME     0
MAN_COLLNAME    0
TYP_INTNAME     0
REL_ROADNAME    0
WRK_ZONENAME    0
LGT_CONDNAME    0
WEATHERNAME

All years of "accident" dataframes only have relevant columns & have no missing values. However, values within columns or column names themselves may need to be adjusted for easier understandability and ease of modeling. 

#### Time Period 1 - Renaming Column Values:

In [52]:
acc_dfs_section_1['1978'].head()

,ST_CASE,PERSONS,HOUR,SCH_BUS,FATALS
0,10001,2,23,0,1
1,10002,4,2,0,2
2,10003,1,1,0,1
3,10004,1,12,0,1
4,10005,1,2,0,1


In [53]:
pd.unique(acc_dfs_section_1['1978']['PERSONS'])

array([ 2,  4,  1,  3,  6,  5,  8,  7,  9, 10, 13, 11, 24, 12, 16, 14, 15,
       17, 18, 19, 34, 32, 49, 29, 23, 22, 25, 38, 33, 74, 20, 28, 26])

Potential issue with the value "99" for HOUR. It could mean it's a missing value:

In [54]:
pd.unique(acc_dfs_section_1['1978']['HOUR'])

array([23,  2,  1, 12, 16,  5,  0,  3, 20,  8, 14, 19,  7, 21, 17, 22, 10,
       11,  9,  4, 15, 18,  6, 13, 24, 99])

In [55]:
pd.unique(acc_dfs_section_1['1978']['SCH_BUS'])

array([0, 1])

In [56]:
pd.unique(acc_dfs_section_1['1978']['FATALS'])

array([1, 2, 3, 4, 5, 6, 7, 0, 8])

All other values in section 1 look reasonable except the 99 for HOUR that was corrected.

#### Time Period 2 - Renaming Column Values:

In [57]:
acc_dfs_section_2['1982'].head()

,ST_CASE,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,2,8,65,2131.0,0,2
1,10002,1,23,0409,0.0,0,1
2,10003,1,12,0234,0.0,0,1
3,10004,4,23,59,99999.0,0,2
4,10005,2,10,0310,0.0,0,1


In [58]:
pd.unique(acc_dfs_section_2['1982']['HOUR'])

array([ 8, 23, 12, 10,  1,  0, 13,  7, 17, 16, 11,  2,  3, 18, 19, 15,  4,
       22, 21,  9,  5, 14, 20,  6, 99, 24])

In [59]:
for key, df in acc_dfs_section_2.items():

    if 'TWAY_ID' in df.columns:
        df.loc[df['TWAY_ID'] == 9999999999, 'TWAY_ID'] = 'Missing'
        acc_dfs_section_2[key] = df

In [60]:
pd.unique(acc_dfs_section_2['1982']['HOUR'])

array([ 8, 23, 12, 10,  1,  0, 13,  7, 17, 16, 11,  2,  3, 18, 19, 15,  4,
       22, 21,  9,  5, 14, 20,  6, 99, 24])

In [61]:
unique_val_counts_MILEPT = acc_dfs_section_2['1982']['MILEPT'].value_counts()

# Sort unique values by their counts in descending order & choose top 10
top_10_values = unique_val_counts_MILEPT.head(10)

print(top_10_values)


0.0        12158
99999.0    10841
1.0          339
2.0          277
3.0          241
4.0          210
5.0          176
10.0         173
7.0          151
8.0          148
Name: MILEPT, dtype: int64


In [62]:
acc_dfs_section_2['1982']['MILEPT'].max()


99999.0

In [63]:
acc_dfs_section_2['1982']['MILEPT'].min()


0.0

The value 99,999 for MILEPT is unrealistic, given how the rest of the values are between 0 and 10. These values will be assumed to be missing & be imputed with the mean of the remaining values:

In [64]:
# Imputing values for unreasonable values for MILEPT (is ~90K):
for year, df in acc_dfs_section_2.items():
    
    avg_milept = round(acc_dfs_section_2[year]['MILEPT'].mean())
    
    acc_dfs_section_2[year].loc[(acc_dfs_section_2[year]['MILEPT'] >= 99990) & 
                                (acc_dfs_section_2[year]['MILEPT'] <= 99999), 
                                'MILEPT'] = avg_milept
    
    acc_dfs_section_2[year]['MILEPT'].fillna(avg_milept, inplace=True)

In [65]:
# Renaming columns for each df in section 2:

for year, df in acc_dfs_section_2.items():
    df.rename(columns={'TWAY_ID': 'ROAD_NAME',
                      'MILEPT': 'MILE_MARKER'}, inplace=True)

#### Time Period 3 - Renaming Column Values:

In [66]:
acc_dfs_section_3['1991'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,2,11,I20,1723,0,1
1,10002,0,6,7,SR13,25,0,2
2,10003,0,1,23,5849,0,0,1
3,10004,0,2,16,2208,0,0,1
4,10005,0,2,15,69,1882,0,1


In [67]:
pd.unique(acc_dfs_section_3['1991']['PEDS'])

array([ 0,  1,  2,  3,  6,  4,  5, 20,  8, 11,  7,  9])

In [68]:
for key, df in acc_dfs_section_3.items():
        
    if 'TWAY_ID' in df.columns:
        df.loc[df['TWAY_ID'] == 9999999999, 'TWAY_ID'] = 'Missing'
        acc_dfs_section_3[key] = df

In [69]:
# Imputing values for unreasonable values for MILEPT (is ~90K):
for year, df in acc_dfs_section_3.items():
    
    avg_milept = round(acc_dfs_section_3[year]['MILEPT'].mean())
    
    acc_dfs_section_3[year].loc[(acc_dfs_section_3[year]['MILEPT'] >= 99990) & 
                                (acc_dfs_section_3[year]['MILEPT'] <= 99999), 
                                'MILEPT'] = avg_milept
    
    acc_dfs_section_3[year]['MILEPT'].fillna(avg_milept, inplace=True)

In [70]:
pd.unique(acc_dfs_section_3['1991']['MILEPT'])

array([1723,   25,    0, ..., 3340,  863, 1192])

In [71]:
# Renaming columns for each df in section 3:

for year, df in acc_dfs_section_3.items():
    df.rename(columns={'PEDS': 'NUM_PEDS',
                      'TWAY_ID': 'ROAD_NAME',
                      'MILEPT': 'MILE_MARKER'}, inplace=True)


#### Time Period 4 - Renaming Column Values:

In [72]:
acc_dfs_section_4['2001'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,1,20,1274,0.0,0,1
1,10002,0,1,5,SR-204,75.0,0,1
2,10003,0,2,22,SR-21,1230.0,0,1
3,10004,0,2,17,I-65,1642.0,0,1
4,10005,0,2,9,US-SR3,2993.0,0,1


In [73]:
acc_dfs_section_4['2005'].head()

,ST_CASE,PEDS,PERSONS,HOUR,TWAY_ID,MILEPT,SCH_BUS,FATALS
0,10001,0,2,16,1172,0.0,0,1
1,10002,0,1,19,SR-69,2275.0,0,1
2,10003,0,1,11,I-20,1567.0,0,1
3,10004,0,1,13,1340,0.0,0,1
4,10005,0,1,5,1491,0.0,0,1


In [74]:
for key, df in acc_dfs_section_4.items():
        
    if 'TWAY_ID' in df.columns:
        df.loc[df['TWAY_ID'] == 9999999999, 'TWAY_ID'] = 'Missing'
        acc_dfs_section_4[key] = df

In [75]:
# Imputing values for unreasonable values for MILEPT (is ~90K):
for year, df in acc_dfs_section_4.items():
    
    avg_milept = round(acc_dfs_section_4[year]['MILEPT'].mean())
    
    acc_dfs_section_4[year].loc[(acc_dfs_section_4[year]['MILEPT'] >= 99990) & 
                                (acc_dfs_section_4[year]['MILEPT'] <= 99999), 
                                'MILEPT'] = avg_milept
    
    acc_dfs_section_4[year]['MILEPT'].fillna(avg_milept, inplace=True)

In [76]:
# Renaming columns for each df in section 4:

for year, df in acc_dfs_section_4.items():
    df.rename(columns={'VE_TOTAL': 'NUM_VEHC',
                       'PEDS': 'NUM_PEDS',
                      'TWAY_ID': 'ROAD_NAME',
                      'MILEPT': 'MILE_MARKER'}, inplace=True)

#### Time Period 5 - Renaming Column Values:

In [77]:
acc_dfs_section_5['2008'].head()

,ST_CASE,PEDS,VE_TOTAL,PERSONS,HOUR,TWAY_ID,MILEPT,LATITUDE,LONGITUD,SCH_BUS,FATALS
0,10001,1,1,3,17,SR-69,0.0,34.326947,-86.489567,0,1
1,10002,0,2,2,12,I-20,1230.0,33.520883,-86.847739,0,1
2,10003,0,1,1,2,1027,0.0,32.477511,-86.346628,0,1
3,10004,0,1,2,10,8438,0.0,30.814786,-88.171917,0,1
4,10005,2,1,3,21,US-SR6,0.0,32.326347,-86.320975,0,2


In [78]:
unique_val_counts_LAT = acc_dfs_section_5['2008']['LATITUDE'].value_counts()

top_10_values = unique_val_counts_LAT.head(10)
top_10_values

36.633871    479
32.266814      3
41.208364      3
36.162686      3
35.493381      3
34.272750      3
35.601975      3
36.808186      3
38.198258      3
38.466492      3
Name: LATITUDE, dtype: int64

In [79]:
acc_dfs_section_5['2008']['LATITUDE'].max()


66.73546111

In [80]:
acc_dfs_section_5['2008']['LATITUDE'].min()


19.04179167

In [81]:
unique_val_counts_LONG = acc_dfs_section_5['2008']['LONGITUD'].value_counts()

top_10_values = unique_val_counts_LONG.head(10)
top_10_values

-91.937022     479
-81.686783       3
-76.555139       3
-85.283517       2
-97.371311       2
-97.185058       2
-85.054436       2
-82.254294       2
-107.791156      2
-82.546861       2
Name: LONGITUD, dtype: int64

In [82]:
acc_dfs_section_5['2008']['LONGITUD'].max()


-67.08131667

In [83]:
acc_dfs_section_5['2008']['LONGITUD'].min()


-170.5122

The values for latitude & longitude look reasonable since all latitude values are between -90 & +90, and all longitude values are between -180 & +180. They don't need to be interpolated. 

Because there are 479 values of latitude at 36.695182 & -91.930549, this point may be of interest for improvement of safety measures for accident prevention.

In [84]:
# KNN imputation function for latitude & longitude missing values that are based
# on location & for which a simple mean may not provide accurate values

def knn_impute(df, columns):
    imputer = KNNImputer(n_neighbors=176)
    df[columns] = imputer.fit_transform(df[columns])
    return df

for key, df in acc_dfs_section_5.items():
    # Replace TWAY_ID values
    if 'TWAY_ID' in df.columns:
        df.loc[df['TWAY_ID'] == 9999999999, 'TWAY_ID'] = 'Missing'
        acc_dfs_section_5[key] = df

    # Replace missing LATITUDE values with NA's first
    if 'LATITUDE' in df.columns:
        values_to_replace = [77.7777, 88.8888, 99.9999]
        df.loc[df['LATITUDE'].isin(values_to_replace), 'LATITUDE'] = np.nan

    # Replace missing LONGITUDE values with NA's first
    if 'LONGITUD' in df.columns:
        values_to_replace = [777.7777, 888.8888, 999.9999]
        df.loc[df['LONGITUD'].isin(values_to_replace), 'LONGITUD'] = np.nan

    # Impute missing LATITUDE and LONGITUDE values with KNN imputation
    if 'LATITUDE' in df.columns and 'LONGITUD' in df.columns:
        columns_to_impute = ['LATITUDE', 'LONGITUD']
        df = knn_impute(df, columns_to_impute)
    
    acc_dfs_section_5[key] = df


In [85]:
# Imputing values for unreasonable values for MILEPT (is ~90K):
for year, df in acc_dfs_section_5.items():
    
    avg_milept = round(acc_dfs_section_5[year]['MILEPT'].mean())
    
    acc_dfs_section_5[year].loc[(acc_dfs_section_5[year]['MILEPT'] >= 99990) & 
                                (acc_dfs_section_5[year]['MILEPT'] <= 99999), 
                                'MILEPT'] = avg_milept
    
    acc_dfs_section_5[year]['MILEPT'].fillna(avg_milept, inplace=True)

In [86]:
# Renaming columns for each df in section 5:

for year, df in acc_dfs_section_5.items():
    df.rename(columns={'VE_TOTAL': 'NUM_VEHC',
                       'PEDS': 'NUM_PEDS',
                      'TWAY_ID': 'ROAD_NAME',
                      'MILEPT': 'MILE_MARKER'}, inplace=True)

#### Time Period 6 - Renaming Column Values:

In [87]:
acc_dfs_section_6['2015'].head()

,STATENAME,ST_CASE,PEDS,VE_TOTAL,PERSONS,MONTHNAME,DAYNAME,DAY_WEEKNAME,HOUR,TWAY_ID,...,MAN_COLLNAME,TYP_INTNAME,REL_ROADNAME,WRK_ZONENAME,LGT_CONDNAME,WEATHERNAME,SCH_BUS,FATALS,COUNTYNAME,CITYNAME
0,Alabama,10001,0,1,1,January,1,Thursday,2,SR-5,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear,0,1,NOT AVAILABLE,NOT AVAILABLE
1,Alabama,10002,0,1,1,January,1,Thursday,22,I-65,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Median,None,Dark - Not Lighted,Cloudy,0,1,NOT AVAILABLE,NOT AVAILABLE
2,Alabama,10003,0,1,2,January,1,Thursday,1,US-SR 6,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear,0,1,NOT AVAILABLE,NOT AVAILABLE
3,Alabama,10004,0,1,1,January,4,Sunday,0,SR-27,...,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Cloudy,0,1,NOT AVAILABLE,NOT AVAILABLE
4,Alabama,10005,0,2,2,January,7,Wednesday,7,US-SR 53,...,Angle,T-Intersection,On Roadway,None,Daylight,Clear,0,1,NOT AVAILABLE,NOT AVAILABLE


In [88]:
acc_dfs_section_6['2015'].iloc[:, 8:24].head()

,HOUR,TWAY_ID,ROUTENAME,RUR_URBNAME,FUNC_SYSNAME,RD_OWNERNAME,MILEPT,LATITUDE,LONGITUD,HARM_EVNAME,MAN_COLLNAME,TYP_INTNAME,REL_ROADNAME,WRK_ZONENAME,LGT_CONDNAME,WEATHERNAME
0,2,SR-5,State Highway,Rural,Principal Arterial - Other,State Highway Agency,1754,33.878653,-87.325328,Embankment,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear
1,22,I-65,Interstate,Rural,Interstate,State Highway Agency,3604,34.910442,-86.908708,Ditch,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Median,None,Dark - Not Lighted,Cloudy
2,1,US-SR 6,U.S. Highway,Rural,Principal Arterial - Other,State Highway Agency,1958,32.142006,-85.758456,Tree (Standing Only),Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Clear
3,0,SR-27,State Highway,Rural,Minor Arterial,State Highway Agency,566,31.439814,-85.510300,Mail Box,Not a Collision with Motor Vehicle In-Transport,Not an Intersection,On Roadside,None,Dark - Not Lighted,Cloudy
4,7,US-SR 53,U.S. Highway,Urban,Principal Arterial - Other,State Highway Agency,308,31.319331,-85.515100,Motor Vehicle In-Transport,Angle,T-Intersection,On Roadway,None,Daylight,Clear


In [89]:
for key, df in acc_dfs_section_6.items():
    # Replace TWAY_ID values
    if 'TWAY_ID' in df.columns:
        df.loc[df['TWAY_ID'] == 9999999999, 'TWAY_ID'] = 'Missing'
        acc_dfs_section_6[key] = df

    # Replace missing LATITUDE values with NA's first
    if 'LATITUDE' in df.columns:
        values_to_replace = [77.7777, 88.8888, 99.9999]
        df.loc[df['LATITUDE'].isin(values_to_replace), 'LATITUDE'] = np.nan

    # Replace missing LONGITUDE values with NA's first
    if 'LONGITUD' in df.columns:
        values_to_replace = [777.7777, 888.8888, 999.9999]
        df.loc[df['LONGITUD'].isin(values_to_replace), 'LONGITUD'] = np.nan

    # Impute missing LATITUDE and LONGITUDE values with KNN imputation
    if 'LATITUDE' in df.columns and 'LONGITUD' in df.columns:
        columns_to_impute = ['LATITUDE', 'LONGITUD']
        df = knn_impute(df, columns_to_impute)
    
    acc_dfs_section_6[key] = df


In [90]:
# Imputing values for unreasonable values for MILEPT (is ~90K):
for year, df in acc_dfs_section_6.items():
    
    avg_milept = round(acc_dfs_section_6[year]['MILEPT'].mean())
    
    acc_dfs_section_6[year].loc[(acc_dfs_section_6[year]['MILEPT'] >= 99990) & 
                                (acc_dfs_section_6[year]['MILEPT'] <= 99999), 
                                'MILEPT'] = avg_milept
    
    acc_dfs_section_6[year]['MILEPT'].fillna(avg_milept, inplace=True)

In [91]:
# Renaming columns for each df in section 6:

for year, df in acc_dfs_section_6.items():
    df.rename(columns={'VE_TOTAL': 'NUM_VEHC',
                       'PEDS': 'NUM_PEDS',
                      'TWAY_ID': 'ROAD_NAME',
                      'MILEPT': 'MILE_MARKER',
                      'MAN_COLLNAME': 'COLLISION_TYPE'}, inplace=True)

Shorten specific values within these variables:

    a. FUNC_SYSNAME (1 or 2 long values)
    
    b. COLLISION_TYPE (1 long value) 
    
    c. HARM_EVNAME (5 or 6 long values)
    
*Will not alter COUNTYNAME because there are many duplicate names but in diff states, so you NEED to keep the number name next to the county name so that you & others can differentiate more easily & don't accidentally get confused in the midst of all this data and many lines of code

In [92]:
for year, df in acc_dfs_section_6.items():
    
    df['FUNC_SYSNAME'] = df['FUNC_SYSNAME'].replace('Principal Arterial - Other Freeways and Expressways', 
                                                    'Principal Arterial - Freeways')
    
    df['COLLISION_TYPE'] = df['COLLISION_TYPE'].replace('The First Harmful Event was Not a Collision with a Motor Vehicle in Transport', 
                                                    'Not a Collision')
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Cargo/Equipment Loss, Shift, or Damage [harmful]', 
                                                    'Cargo/Equipment Movement')
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Motor Vehicle in Motion Outside the Trafficway', 
                                                    'Motion Off Trafficway')
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Motor Vehicle In-Transport Strikes or is Struck by Cargo, Persons or Objects Set-in-Motion from/by Another Motor Vehicle In Transport', 
                                                    'Vehicle Strikes Cargo from Other Vehicle')
    
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Object That Had Fallen From Motor Vehicle In-Transport', 
                                                  'Object Fell from Moving Vehicle')
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Pavement Surface Irregularity (Ruts, Potholes, Grates, etc.)', 
                                                  'Road Ruts, Potholes, or Grates')
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Object That Had Fallen From Motor Vehicle In-Transport', 
                                                  'Object Fell from Moving Vehicle')
    
    df['HARM_EVNAME'] = df['HARM_EVNAME'].replace('Ridden Animal or Animal Drawn Conveyance', 
                                                  'Animal-Drawn Vehicles')
    

In [93]:
column_to_check = 'HARM_EVNAME'
value_to_match = 'Object Fell from Moving Vehicle'

# Filter rows where the specified column equals the specified value
filtered_df = df[df[column_to_check] == value_to_match]

# Print the filtered DataFrame
print("Rows where column '{}' equals '{}':".format(column_to_check, value_to_match))
print(filtered_df)


Rows where column 'HARM_EVNAME' equals 'Object Fell from Moving Vehicle':
       STATENAME  ST_CASE  NUM_PEDS  NUM_VEHC  PERSONS    COUNTYNAME  \
11488    Georgia   130338         0         1        1  CARROLL (45)   
11963    Georgia   130839         0         1        1   COWETA (77)   
12966      Idaho   160039         0         2        4   JEROME (53)   
31932  Tennessee   470546         0         2        2     KNOX (93)   
35577      Texas   483006         0         2        2    HUNT (231)   
37355   Virginia   510293         0         2        2   AMHERST (9)   

             CITYNAME  MONTHNAME  DAYNAME DAY_WEEKNAME  ...    LONGITUD  \
11488  NOT APPLICABLE      March       26       Friday  ...  -84.964183   
11963    Not Reported       July        9       Friday  ...  -84.754969   
12966  NOT APPLICABLE      April        2       Friday  ... -114.472300   
31932       KNOXVILLE       June       10     Thursday  ...  -83.986886   
35577  NOT APPLICABLE  September       18     

#### Exporting data to CSV files to model in a separate notebook:

In [94]:
# Concatenate all years into 1 df & add a 'Year' column:
comb_df_sect_1 = pd.concat([df.assign(YEAR=name) for name, df in acc_dfs_section_1.items()], ignore_index=True)

# Puts YEAR column as first column of df:
comb_df_sect_1 = comb_df_sect_1[['YEAR'] + [col for col in comb_df_sect_1.columns if col != 'YEAR']]

# Write the combined df to a CSV file:
comb_df_sect_1.to_csv('acc_sect_1.csv', index=False)


In [95]:
comb_df_sect_2 = pd.concat([df.assign(YEAR=name) for name, df in acc_dfs_section_2.items()], ignore_index=True)

comb_df_sect_2 = comb_df_sect_2[['YEAR'] + [col for col in comb_df_sect_2.columns if col != 'YEAR']]

comb_df_sect_2.to_csv('acc_sect_2.csv', index=False)

In [96]:
comb_df_sect_3 = pd.concat([df.assign(YEAR=name) for name, df in acc_dfs_section_3.items()], ignore_index=True)

comb_df_sect_3 = comb_df_sect_3[['YEAR'] + [col for col in comb_df_sect_3.columns if col != 'YEAR']]

comb_df_sect_3.to_csv('acc_sect_3.csv', index=False)

In [97]:
comb_df_sect_4 = pd.concat([df.assign(YEAR=name) for name, df in acc_dfs_section_4.items()], ignore_index=True)

comb_df_sect_4 = comb_df_sect_4[['YEAR'] + [col for col in comb_df_sect_4.columns if col != 'YEAR']]

comb_df_sect_4.to_csv('acc_sect_4.csv', index=False)

In [98]:
comb_df_sect_5 = pd.concat([df.assign(YEAR=name) for name, df in acc_dfs_section_5.items()], ignore_index=False)

comb_df_sect_5 = comb_df_sect_5[['YEAR'] + [col for col in comb_df_sect_5.columns if col != 'YEAR']]

comb_df_sect_5.to_csv('acc_sect_5.csv', index=False)

In [99]:
comb_df_sect_6 = pd.concat([df.assign(YEAR=name) for name, df in acc_dfs_section_6.items()], ignore_index=False)

comb_df_sect_6 = comb_df_sect_6[['YEAR'] + [col for col in comb_df_sect_6.columns if col != 'YEAR']]

comb_df_sect_6.to_csv('acc_sect_6.csv', index=False)